<a href="https://colab.research.google.com/github/GSylph/Financial-Agent-Chatbot/blob/main/A2A_MCP_Financial_Agent_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A2A + MCP + LangChain **Multi - Agent Chatbot** to provide comprehensive stock market insights

##Install all libraries


In [1]:
!pip install  langchain python-a2a yfinance pandas requests beautifulsoup4 --quiet
!pip install google-generativeai "langchain-google-genai"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.0/395.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00


In [2]:
import os
import re
import yfinance
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# LangChain + Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType

# A2A
from python_a2a import OpenAIA2AServer, run_server
from python_a2a import AgentCard, AgentSkill
from python_a2a.mcp import FastMCP
from python_a2a.langchain import to_langchain_agent, to_langchain_tool

# Set Gemini API key
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY


In [3]:
##Define the Financial Agent's capabilities

agent_card=AgentCard(
    name="Stock Market Expert",
    description="Expert in market trends, fundamentals and investment strategies.",
    url="http://localhost:5000",
    version="1.0.0",
    skills=[
        AgentSkill(
            name="Market Analysis",
            description="Analyze overall market sentiment and key indicators.",
            examples=["Whats the current market sentiment?","Impact of interest rates on tech stocks?"]
        ),
        AgentSkill(
            name="Investment Strategies",
            description="Describe risk management and portfolio diversification",
            examples=["How to diversify my portfolio?","Explanation of dollar-cost averaging"]
        ),
        AgentSkill(
            name="Company Analysis",
            description="Interpret financial ratios and company fundamentals",
            examples=["How to read P/E ratios?", "Key metrics for evaluation of growth stocks"]
        )
    ]
)
print("Agent profile created successfully")
print(f"\nAgent name: {agent_card.name}")
print(f"\nSkills: {[skill.name for skill in agent_card.skills]}")

Agent profile created successfully

Agent name: Stock Market Expert

Skills: ['Market Analysis', 'Investment Strategies', 'Company Analysis']


##Create the A2A instance

In [4]:
a2a_server = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3
)

print("A2A Server initialized successfully with Gemini!")


A2A Server initialized successfully with Gemini!


In [5]:
# import threading
# import time

# def start_a2a_server():
#   """Start a2a server in the background thread"""
#   try:
#     run_server(a2a_server,host='0.0.0.0',port=5000)
#   except Exception as e:
#     print(f"A2A Server error: {e}")

# #Start server in the backgounrd thread
# a2a_thread = threading.Thread(target=start_a2a_server, daemon=True)
# a2a_thread.start()

# # Give server time to start
# time.sleep(3)
# print("A2A Server started on http://localhost:5000")

##Create MCP server for financial tools


In [5]:
# Create MCP server for financial tools
mcp_server = FastMCP(
    name="FinanceTools",
    description="Tools for retrieving stock data and financial news."
)

print("MCP Server initialized!")


MCP Server initialized!


In [6]:
@mcp_server.tool(
    name="stock_data",
    description="Fetch current metrics and data for stocks by ticker symbols or company names."
)
def stock_data(input_str=None, **kwargs):
  """Fetch stock data using yfinance"""
  input_str=kwargs.get('input',input_str)
  if not input_str:
    return {"error": "No input provided."}
  # Extract ticker symbols
  tickers=[]
  if ',' in input_str:
    tickers = [t.strip().upper() for t in input_str.split(',')]
  else:
    tickers = [w.upper() for w in re.findall(r"\b[A-Za-z]{1,5}\b", input_str)]
  # Handle common company names
  common_names = {
    'apple': 'AAPL',
    'nvidia': 'NVDA',
    'microsoft': 'MSFT',
    'google': 'GOOGL',
    'amazon': 'AMZN',
    'tesla': 'TSLA'
  }
  if not tickers:
    for name, ticker in common_names.items():
      if name in input_str.lower():
        tickers.append(ticker)

  results={}

  for ticker in tickers:
    try:
      tk=yfinance.Ticker(ticker)
      hist=tk.history(period="1mo")

      if hist.empty:
        results[ticker]={"error": "No data available."}
        continue

      first_day=hist.iloc[0]
      last_day=hist.iloc[-1]
      price_change=float(last_day['Close']-first_day['Close'])
      pct_change=float(price_change/first_day['Close'])*100

      info=tk.info

      summary = {
        "latest_price": float(last_day['Close']),
        "price_change": price_change,
        "percent_change": pct_change,
        "52_week_high": info.get('fiftyTwoWeekHigh'),
        "52_week_low": info.get('fiftyTwoWeekLow'),
        "market_cap": info.get('marketCap'),
        "pe_ratio": info.get('trailingPE'),
        "volume": int(last_day['Volume'])
      }
      results[ticker]=summary
    except Exception as e:
      results[ticker]={"error":f"Failed to fetch data: {str(e)}"}
  return results

print("Stock data fetcher tool created!")


Stock data fetcher tool created!


In [7]:
@mcp_server.tool(
    name="web_scraper",
    description="Scrape latest financial headlines and company snapshot from Finviz."
)
def web_scraper(input_str=None, **kwargs):
  """Scrape latest financial news and data from Finviz"""
  ticker=(kwargs.get('input') or input_str or '').strip().upper()
  if not ticker:
    return {"error": "No ticker symbol provided."}
  url = f"https://finviz.com/quote.ashx?t={ticker.lower()}"
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

  try:
    response=requests.get(url, headers=headers)
    soup=BeautifulSoup(response.text,'html.parser')

    #Extract news items
    news_items = []
    news_table = soup.find('table', {'id': 'news-table'})
    if news_table:
      for row in news_table.find_all('tr')[:5]:  # Get top 5 news items
        cells = row.find_all('td')
        if len(cells) >= 2:
          date_cell = cells[0]
          title_cell = cells[1]
          link_tag = title_cell.find('a')
          if link_tag:
            title = link_tag.text.strip()
            link = link_tag.get('href', '')
            if link and not link.startswith('http'):
              link = urljoin(url, link)

            news_items.append({
              "date": date_cell.text.strip(),
              "title": title,
              "link": link
            })
    # Extract company snapshot data
    snapshot_details = {}
    snapshot_table = soup.find('table', {'class': 'snapshot-table2'})
    if snapshot_table:
      for row in snapshot_table.find_all('tr'):
        cells = row.find_all('td')
        for i in range(0, len(cells), 2):
          if i + 1 < len(cells):
            key = cells[i].text.strip()
            value = cells[i + 1].text.strip()
            snapshot_details[key] = value
    return {
      "ticker": ticker,
      "news_items": news_items,
      "snapshot": snapshot_details
    }

  except Exception as e:
    return {"error": f"Failed to scrape data: {str(e)}"}
print("Financial news scraper tool created!")

Financial news scraper tool created!


In [9]:
# def start_mcp_server():
#     """Start MCP server in background thread"""
#     try:
#         mcp_server.run(host='0.0.0.0', port=6000)
#     except Exception as e:
#         print(f"MCP Server error: {e}")

# # Start MCP server in background thread
# mcp_thread = threading.Thread(target=start_mcp_server, daemon=True)
# mcp_thread.start()

# # Give server time to start
# time.sleep(3)
# print("MCP Server started on http://localhost:6000")

##Convert both A2A agent & MCP tools to LangChain


In [10]:
# # Convert A2A agent to LangChain
# try:
#   a2a_agent=to_langchain_agent("http://localhost:5000")
#   print("A2A agent converted to LangChain successfully!")
# except Exception as e:
#   print(f"Error converting A2A agent: {e}")

# # Convert MCP tools to LangChain
# try:
#   stock_tool=to_langchain_agent("http://localhost:6000/stock_data")
#   news_tool=to_langchain_agent("http://localhost:6000/web_scraper")
#   print("MCP tools converted to LangChain successfully!")
# except Exception as e:
#   print(f"Error converting MCP tools: {e}")



In [8]:
def ask_expert(query):
  """Ask the financial expert using direct LLM call"""
  try:
    prompt = f"You are a financial expert. Answer this question: {query}"
    result = llm.invoke(prompt)
    return result.content
  except Exception as e:
    return f"Error asking expert: {str(e)}"

def fetch_stock_data(query):
  """Fetch stock data using direct function call"""
  try:
    return stock_data(input_str=query)
  except Exception as e:
    return f"Error fetching stock data: {str(e)}"

def fetch_financial_news(query):
  """Fetch financial news using direct function call"""
  try:
    return web_scraper(input_str=query)
  except Exception as e:
    return f"Error fetching news: {str(e)}"

##Create LangChain Tool objects


In [9]:
tools=[
    Tool(
        name="StockExpert",
        func=ask_expert,
        description="Ask financial questions to a stock market expert. Use for market analysis, investment strategies, and financial advice."
    ),
    Tool(
        name="StockData",
        func=fetch_stock_data,
        description="Retrieve current stock metrics and data. Input should be ticker symbols or company names."
    ),
    Tool(
        name="FinancialNews",
        func=fetch_financial_news,
        description="Get latest financial headlines and company snapshots. Input should be a ticker symbol."
    )
]

##Initialize the main LLM & meta-agent


In [10]:
# Initialize the main Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3
)
# Create the meta-agent
meta_agent = initialize_agent(
    tools=tools,  # Your list of LangChain-compatible tools
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate"
)

print("Meta-agent initialized successfully with Gemini!")


Meta-agent initialized successfully with Gemini!


<ipython-input-10-901746d66a79>:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  meta_agent = initialize_agent(


##Test both Test stock data fetcher & news scraper


In [11]:
# Test stock data fetcher
print("Testing stock data fetcher...")
test_stock_data = stock_data("AAPL,NVDA")
print("Stock Data Result:", test_stock_data)
print("\n" + "="*50 + "\n")

# Test news scraper
print("Testing news scraper...")
test_news_data = web_scraper("AAPL")
print("News Data Result:", test_news_data)
print("\n" + "="*50 + "\n")

Testing stock data fetcher...
Stock Data Result: {'AAPL': {'latest_price': 201.4499969482422, 'price_change': -9.339996337890625, 'percent_change': -4.43094863863496, '52_week_high': 260.1, '52_week_low': 169.21, 'market_cap': 3045722423296, 'pe_ratio': 31.329704, 'volume': 72726700}, 'NVDA': {'latest_price': 142.6300048828125, 'price_change': 19.6300048828125, 'percent_change': 15.959353563262196, '52_week_high': 153.13, '52_week_low': 86.62, 'market_cap': 3478403678208, 'pe_ratio': 46.00968, 'volume': 184612200}}


Testing news scraper...
News Data Result: {'ticker': 'AAPL', 'news_items': [{'date': 'Today 12:01AM', 'title': 'iOS 26Visual Intelligence', 'link': 'https://finance.yahoo.com/video/ios-26-visual-intelligence-040100329.html'}, {'date': 'Jun-09-25 11:57PM', 'title': 'Huawei ramps up in-house tech with foldable PC, Pura 80 smartphone to challenge Apple', 'link': 'https://www.digitimes.com/news/a20250610PD212/huawei-foldable-pc-apple-smartphone.html'}, {'date': '09:50PM', 'tit

##Test various types of queries


In [15]:
# Test various types of queries
test_queries = [
  "What’s the market outlook for Tesla this week?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {query}")
    print('='*60)

    try:
        response = meta_agent.invoke(query)
        print(response['output'])
    except Exception as e:
        print(f"Error: {e}")

    print('='*60)


TEST QUERY 1: What’s the market outlook for Tesla this week?


> Entering new AgentExecutor chain...
I need to get a sense of what the market thinks about Tesla's prospects for the week. I should consult a stock market expert to get their analysis.
Action: StockExpert
Action Input: What is the market outlook for Tesla this week?
Observation: Okay, let's break down the market outlook for Tesla (TSLA) this week.  It's important to remember that the stock market is inherently unpredictable, and this is *not* financial advice. This is an analysis based on publicly available information and common market factors.

**Factors that Could Positively Influence Tesla's Stock This Week:**

*   **Strong Delivery Numbers (If Released):** If Tesla releases strong delivery numbers for the previous quarter (which often happens at the beginning of a new quarter), it could provide a significant boost to the stock. Investors closely watch these figures as a key indicator of demand and production efficien

In [13]:
def interactive_query_tester(meta_agent):
    """
    Interactive testing function for meta_agent

    Args:
        meta_agent: Your configured meta agent instance
    """
    print("=" * 60)
    print("🤖 INTERACTIVE META AGENT TESTER")
    print("=" * 60)
    print("Enter your queries below. Type 'quit', 'exit', or 'q' to stop.")
    print("Type 'help' for available commands.")
    print("-" * 60)

    query_count = 0

    while True:
        try:
            # Get user input
            query = input("\n💬 Enter your query: ").strip()
            # Process the query
            query_count += 1
            print(f"\n{'=' * 60}")
            print(f"🔍 QUERY #{query_count}: {query}")
            print('=' * 60)

            # Execute query with error handling
            try:
                response = meta_agent.invoke(query)

                # Handle different response formats
                if isinstance(response, dict):
                    if 'output' in response:
                        print("📤 RESPONSE:")
                        print(response['output'])
                    else:
                        print("📤 FULL RESPONSE:")
                        print(response)
                else:
                    print("📤 RESPONSE:")
                    print(response)

            except Exception as e:
                print(f"❌ ERROR: {e}")
                print(f"Error type: {type(e).__name__}")

            print('=' * 60)

        except KeyboardInterrupt:
            print("\n\n⏹️  Interrupted by user. Exiting...")
            break
        except EOFError:
            print("\n\n👋 Input ended. Exiting...")
            break
        except Exception as e:
            print(f"\n❌ Unexpected error: {e}")
            continue

In [14]:
interactive_query_tester(meta_agent)


🤖 INTERACTIVE META AGENT TESTER
Enter your queries below. Type 'quit', 'exit', or 'q' to stop.
Type 'help' for available commands.
------------------------------------------------------------

💬 Enter your query: Whats the latest updates in Apple stocks?

🔍 QUERY #1: Whats the latest updates in Apple stocks?


> Entering new AgentExecutor chain...
I need to find the latest news and updates regarding Apple stocks.
Action: FinancialNews
Action Input: AAPL
Observation: {'ticker': 'AAPL', 'news_items': [{'date': 'Today 01:00AM', 'title': 'Apples AI event falls flat as iPhone maker struggles', 'link': 'https://finance.yahoo.com/m/ad88e673-5b53-34b6-847d-717ee1011be7/apple%E2%80%99s-ai-event-falls-flat.html'}, {'date': '12:01AM', 'title': 'iOS 26Visual Intelligence', 'link': 'https://finance.yahoo.com/video/ios-26-visual-intelligence-040100329.html'}, {'date': 'Jun-09-25 11:57PM', 'title': 'Huawei ramps up in-house tech with foldable PC, Pura 80 smartphone to challenge Apple', 'link': 'https

ERROR:yfinance:$NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")



Observation: {'NS': {'error': 'No data available.'}}
Thought:I am unable to retrieve data for Reliance using the available tools. I will try asking the stock expert directly.
Action: StockExpert
Action Input: Should I buy Reliance stocks?
Observation: Okay, let's break down whether or not you should buy Reliance stocks.  As a financial expert, I can't give you a definitive "yes" or "no" because investment decisions depend entirely on your individual circumstances. However, I can provide a comprehensive analysis to help you make an informed decision.

**Here's a framework to consider:**

**I. Understanding Reliance Industries (RIL):**

*   **Business Overview:** Reliance Industries is a massive conglomerate with diverse interests spanning:
    *   **Oil and Gas (O&G):**  Traditional strength, but increasingly focusing on cleaner energy.
    *   **Petrochemicals:**  Manufacturing of polymers and other chemical products.
    *   **Retail:** Reliance Retail is a dominant player in India's

ERROR:yfinance:$PALO ALTO NETWORKS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$CROWDSTRIKE: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$SOLAREDGE: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$ENPHASE ENERGY: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$TESLA: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")



Observation: {'NVIDIA': {'error': 'Failed to fetch data: HTTP Error 404: '}, 'AMD': {'latest_price': 121.7300033569336, 'price_change': 13.610000610351562, 'percent_change': 12.587865579555594, '52_week_high': 187.28, '52_week_low': 76.48, 'market_cap': 197373034496, 'pe_ratio': 88.210144, 'volume': 55309300}, 'PALO ALTO NETWORKS': {'error': 'No data available.'}, 'CROWDSTRIKE': {'error': 'No data available.'}, 'SOLAREDGE': {'error': 'No data available.'}, 'ENPHASE ENERGY': {'error': 'No data available.'}, 'TESLA': {'error': 'No data available.'}, 'BYD': {'latest_price': 73.93000030517578, 'price_change': -0.8499984741210938, 'percent_change': -1.1366655362348295, '52_week_high': 80.5, '52_week_low': 51.62, 'market_cap': 6018131456, 'pe_ratio': 12.119673, 'volume': 725300}}
Thought:The StockData tool returned errors for many of the stocks. I will try FinancialNews to get some snapshots of the companies that returned errors.

Action: FinancialNews
Action Input: NVIDIA, Palo Alto Networ